在沙盒环境，进行期货组合优化及回测

In [1]:
import os,pdb,itertools,copy,datetime
os.environ['ULTRON_DATA'] = 'keim'

In [2]:
import pandas as pd
import numpy as np
from ultron.env import *
from ultron.strategy.experimental.multiple_factor import MultipleFactor
from ultron.strategy.fut_strategy import create_params
from ultron.strategy.strategy import StrategyEngine

/var/log/ultron/2022-09-27.log


In [3]:
enable_example_env()

2022-09-27 19:06:32,655 - [env.py:67] - ultron - INFO - enable example env will only read /home/kerry/ultron/rom/sandbox/keim


#### 加载行情数据

In [4]:
market_data = pd.read_csv(os.path.join(g_project_data, 'market_data.csv'), index_col=0)
market_data['trade_date'] = pd.to_datetime(market_data['trade_date'])
market_data.head()

,trade_date,code,openPrice,highestPrice,lowestPrice,closePrice,turnoverVol
0,2017-10-27,A,4462.578191,4463.801485,4413.646412,4435.665713,158774
1,2017-10-27,AL,15625.658581,15658.904663,15430.931529,15449.929290,293630
2,2017-10-27,BU,3310.339921,3336.950371,3283.729472,3302.356787,461826
3,2017-10-27,C,2009.751001,2014.561895,2001.331936,2002.534660,375480
4,2017-10-27,CF,20517.496003,20531.174333,20408.069357,20449.104349,84032


#### 加载信号值

In [5]:
signal_data = pd.read_csv(os.path.join(g_project_data, 'signal.csv'), index_col=0)
signal_data['trade_date'] = pd.to_datetime(signal_data['trade_date'])
signal_data.head()

,trade_date,code,BM_MainFar_80D_23_5_5,BM_MainFar_80D_27_5_5,BM_MainFar_80D_25_5_5,BM_RecentFar_20D_5_1_5,BM_RecentFar_40D_9_2_3,BM_RecentFar_40D_3_1_3,BM_RecentFar_40D_11_2_3,BM_RecentFar_40D_13_1_3,...,T_DnVolatility_1_10D_21_1_5,T_DnVolatility_2_20D_25_1_4,T_DnVolatility_2_20D_27_1_4,WeightNetIntTotalChg5D_11_5_3,WeightNetIntTotalChg5D_13_5_3,WeightShortVolRelTotIntChg_9_4_4,inventory_5_3_5,inventory_10_1_5,inventory_15_4_5,profitratio_5_3_5
0,2017-11-01,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-11-01,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-11-01,BU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-11-01,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-11-01,CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
columns = [col for col in signal_data.columns if col not in ['trade_date','code']]

#### 等权合并信号

In [7]:
mixed_signal = copy.deepcopy(signal_data)
mixed_signal['signal'] = np.nanmean(mixed_signal.set_index(
    ['trade_date','code'])[columns],axis=1)
mixed_signal = mixed_signal[['trade_date','code','signal']]
mixed_signal = mixed_signal.dropna(subset=['signal'])
mixed_signal.head()

,trade_date,code,signal
0,2017-11-01,A,0.0
1,2017-11-01,AL,1.0
2,2017-11-01,BU,-1.0
3,2017-11-01,C,1.0
4,2017-11-01,CF,1.0


#### 前置收益率计算

In [8]:
def prev_returs_impl(price_data, key, name):
    price_tb = price_data[key].unstack()
    price_tb.fillna(method='pad', inplace=True)
    return_tb = np.log(price_tb / price_tb.shift(1))
    return_tb = return_tb.replace([np.inf, -np.inf], np.nan)
    return_tb = return_tb.stack().reindex(price_data.index)
    return_tb.name = name
    return return_tb

#### 后置收益率计算

In [9]:
def next_returs_impl(price_data, key, name):
    price_tb = price_data[key].unstack()
    price_tb.fillna(method='pad', inplace=True)
    return_tb = np.log(price_tb.shift(-1) / price_tb)
    return_tb = return_tb.replace([np.inf, -np.inf], np.nan)
    return_tb = return_tb.stack().reindex(price_data.index)
    return_tb.name = name
    return return_tb

In [10]:
next_rets = next_returs_impl(
    market_data.set_index(['trade_date','code']),'closePrice','nxt1_ret').reset_index()
next_rets['trade_date'] = pd.to_datetime(next_rets['trade_date'])
next_rets.head()

,trade_date,code,nxt1_ret
0,2017-10-27,A,-0.000276
1,2017-10-27,AL,0.001222
2,2017-10-27,BU,0.019945
3,2017-10-27,C,-0.005420
4,2017-10-27,CF,0.001003


In [11]:
prev_rets = prev_returs_impl(
    market_data.set_index(['trade_date','code']),'closePrice','pev1_ret').reset_index()
prev_rets['trade_date'] = pd.to_datetime(prev_rets['trade_date'])
prev_rets.tail()

,trade_date,code,pev1_ret
37248,2022-06-22,TA,-0.020493
37249,2022-06-22,V,-0.035797
37250,2022-06-22,Y,-0.040992
37251,2022-06-22,ZC,0.000237
37252,2022-06-22,ZN,-0.002764


##### 因子加权组合

In [12]:
weighted  = MultipleFactor(signal_data=signal_data, 
                    volatility_data=None, 
                    returns_data=None)

In [13]:
weighted_res = weighted.equal_run(columns=columns, equal=0)
weighted_res.head()

2022-09-27 19:06:44,202 - [multiple_factor.py:227] - ultron - INFO - starting construction


,trade_date,weight,code
0,2017-11-01,0.090909,MA
1,2017-11-01,0.090909,AL
2,2017-11-01,0.090909,C
3,2017-11-01,0.090909,CF
4,2017-11-01,0.090909,CS


#### 加载行业信息

In [14]:
industry_data = pd.read_csv(os.path.join(g_project_data, 'industry_data.csv'), index_col=0)
industry_data.head()

,code,industry
0,RB,black_series
1,I,black_series
2,ZC,black_series
3,JM,black_series
4,HC,black_series


#### 初始权重为因子加权权重

In [15]:
benchmark_weighed = copy.deepcopy(weighted_res)
benchmark_weighed['trade_date'] = pd.to_datetime(benchmark_weighed['trade_date'])
benchmark_weighed.head()

,trade_date,weight,code
0,2017-11-01,0.090909,MA
1,2017-11-01,0.090909,AL
2,2017-11-01,0.090909,C
3,2017-11-01,0.090909,CF
4,2017-11-01,0.090909,CS


##### 组合优化

In [16]:
factors_data = mixed_signal.merge(prev_rets, on=['trade_date','code']).merge(
    next_rets, on=['trade_date','code']).merge(
    industry_data, on=['code']).set_index(['trade_date','code'])
factors_data.head()

,,signal,pev1_ret,nxt1_ret,industry
trade_date,code,,,,
2017-11-01,A,0.000,-0.000275,0.002201,agricultural
2017-11-02,A,0.000,0.002201,0.001648,agricultural
2017-11-03,A,0.000,0.001648,0.001645,agricultural
2017-11-06,A,0.000,0.001645,0.004919,agricultural
2017-11-07,A,-0.125,0.004919,-0.000818,agricultural


In [17]:
industry_dummy = pd.get_dummies(factors_data['industry'])
industry_dummy.head()

,,agricultural,black_series,chemicals,non_ferrous
trade_date,code,,,,
2017-11-01,A,1,0,0,0
2017-11-02,A,1,0,0,0
2017-11-03,A,1,0,0,0
2017-11-06,A,1,0,0,0
2017-11-07,A,1,0,0,0


In [18]:
factors_data = pd.concat([factors_data,industry_dummy],axis=1).sort_values(
    by=['trade_date','code'],ascending=True
)
factors_data = factors_data.reset_index()
factors_data.tail()

,trade_date,code,signal,pev1_ret,nxt1_ret,industry,agricultural,black_series,chemicals,non_ferrous
37145,2022-06-22,TA,-0.200000,-0.020493,NaN,chemicals,0,0,1,0
37146,2022-06-22,V,-0.033333,-0.035797,NaN,chemicals,0,0,1,0
37147,2022-06-22,Y,-0.250000,-0.040992,NaN,agricultural,1,0,0,0
37148,2022-06-22,ZC,-0.105263,0.000237,NaN,black_series,0,1,0,0
37149,2022-06-22,ZN,0.166667,-0.002764,NaN,non_ferrous,0,0,0,1


In [19]:
factor_col = ['signal']
diff_cols = [col for col in factors_data.columns if col not in factor_col]
neutralized_styles = factors_data['industry'].unique().tolist()

In [20]:
factors_data = factors_data.merge(
    benchmark_weighed, on=['trade_date','code']).sort_values(
    by=['trade_date','code'],ascending=True
)
factors_data = factors_data.rename(
    columns={'industry':'industry_code','signal':'factor'})

is_in_benchmark = (factors_data.weight != 0.).astype(float).values.reshape((-1, 1))
factors_data.loc[:, 'benchmark'] = is_in_benchmark
factors_data.loc[:, 'total'] = np.ones_like(is_in_benchmark)
factors_data.tail()

,trade_date,code,factor,pev1_ret,nxt1_ret,industry_code,agricultural,black_series,chemicals,non_ferrous,weight,benchmark,total
37145,2022-06-22,TA,-0.200000,-0.020493,NaN,chemicals,0,0,1,0,-0.093065,1.0,1.0
37146,2022-06-22,V,-0.033333,-0.035797,NaN,chemicals,0,0,1,0,-0.015511,1.0,1.0
37147,2022-06-22,Y,-0.250000,-0.040992,NaN,agricultural,1,0,0,0,-0.116331,1.0,1.0
37148,2022-06-22,ZC,-0.105263,0.000237,NaN,black_series,0,1,0,0,-0.048981,1.0,1.0
37149,2022-06-22,ZN,0.166667,-0.002764,NaN,non_ferrous,0,0,0,1,0.062500,1.0,1.0


In [21]:
effective_codes = factors_data.industry_code.unique().tolist() ### 有效行业
invalid_codes = [] ### 无效行业
target_vol = 0.05 ### 目标波动率
turn_over_target = 0.4  ### 换手率
lbound = -0.1 ### 单个标的优化下限
ubound = 0.1 ### 单个标的优化上限
benchmark_lower = -1.00 ## 基于基准下限
benchmark_upper = 1.001 ## 基于基准上限
total_lower = -0.01 ## 组合下限
total_upper = 0.01 ## 组合上限
effective_industry_lower = -0.1 ## 有效行业下限
effective_industry_upper = 0.1 ## 有效行业上限
invalid_industry_lower = -0.2 ## 无效行业下限
invalid_industry_lower = 0.2 ## 无效行业上限
cov_method = 'unshrunk' ## 协方差收益率模式
method = 'longshort_mean_variance'

In [22]:
params = create_params(
    effective_codes=effective_codes, invalid_codes=invalid_codes,
    target_vol=target_vol,turn_over_target=turn_over_target, 
    lbound=lbound, ubound=ubound,
    benchmark_lower=benchmark_lower,benchmark_upper=benchmark_upper,
    total_lower=total_lower,total_upper=total_upper,
    effective_industry_lower=effective_industry_lower,
    effective_industry_upper=effective_industry_upper,
    method=method,
    is_benchmark=1)

#### 创建策略实例

In [23]:
strategy = StrategyEngine.create_class(name='fut')(alpha_model=None, total_data=factors_data, 
                    start_date=datetime.datetime(2021,1,4))

##### 构建权重

In [24]:
positions = strategy.create_positions(params=params)

2022-09-27 19:06:58,321 - [fut_strategy.py:312] - ultron - INFO - starting re-balance ...
2022-09-27 19:06:58,331 - [fut_strategy.py:330] - ultron - INFO - running setting finished ...
2022-09-27 19:06:58,364 - [fut_strategy.py:169] - ultron - INFO - alpha models training finished ...
2022-09-27 19:06:59,185 - [fut_strategy.py:373] - ultron - INFO - 2021-01-04 00:00:00 re-balance: 34 codes
2022-09-27 19:06:59,220 - [fut_strategy.py:303] - ultron - ERROR - 2021-01-04T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:06:59,230 - [fut_strategy.py:373] - ultron - INFO - 2021-01-05 00:00:00 re-balance: 34 codes
2022-09-27 19:06:59,264 - [fut_strategy.py:303] - ultron - ERROR - 2021-01-05T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:06:59,274 - [fut_strategy.py:373] - ultron - INFO - 2021-01-06 00:00:00 re-balance: 34 codes
2022-09-27 19:06:59,308 - [fut_strategy.py:303] - ultron - ERROR - 2021-01-06T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:06:5

2022-09-27 19:07:02,166 - [fut_strategy.py:373] - ultron - INFO - 2021-04-12 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,211 - [fut_strategy.py:373] - ultron - INFO - 2021-04-13 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,255 - [fut_strategy.py:373] - ultron - INFO - 2021-04-14 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,305 - [fut_strategy.py:373] - ultron - INFO - 2021-04-15 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,351 - [fut_strategy.py:373] - ultron - INFO - 2021-04-16 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,396 - [fut_strategy.py:373] - ultron - INFO - 2021-04-19 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,440 - [fut_strategy.py:373] - ultron - INFO - 2021-04-20 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,483 - [fut_strategy.py:373] - ultron - INFO - 2021-04-21 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,527 - [fut_strategy.py:373] - ultron - INFO - 2021-04-22 00:00:00 re-balance: 35 codes
2022-09-27 19:07:02,572 - [fut_strate

2022-09-27 19:07:05,777 - [fut_strategy.py:373] - ultron - INFO - 2021-08-03 00:00:00 re-balance: 35 codes
2022-09-27 19:07:05,824 - [fut_strategy.py:373] - ultron - INFO - 2021-08-04 00:00:00 re-balance: 35 codes
2022-09-27 19:07:05,869 - [fut_strategy.py:373] - ultron - INFO - 2021-08-05 00:00:00 re-balance: 35 codes
2022-09-27 19:07:05,920 - [fut_strategy.py:373] - ultron - INFO - 2021-08-06 00:00:00 re-balance: 35 codes
2022-09-27 19:07:05,966 - [fut_strategy.py:373] - ultron - INFO - 2021-08-09 00:00:00 re-balance: 35 codes
2022-09-27 19:07:06,013 - [fut_strategy.py:373] - ultron - INFO - 2021-08-10 00:00:00 re-balance: 35 codes
2022-09-27 19:07:06,060 - [fut_strategy.py:373] - ultron - INFO - 2021-08-11 00:00:00 re-balance: 35 codes
2022-09-27 19:07:06,105 - [fut_strategy.py:373] - ultron - INFO - 2021-08-12 00:00:00 re-balance: 35 codes
2022-09-27 19:07:06,153 - [fut_strategy.py:373] - ultron - INFO - 2021-08-13 00:00:00 re-balance: 35 codes
2022-09-27 19:07:06,199 - [fut_strate

2022-09-27 19:07:09,380 - [fut_strategy.py:373] - ultron - INFO - 2021-11-29 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,425 - [fut_strategy.py:373] - ultron - INFO - 2021-11-30 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,470 - [fut_strategy.py:373] - ultron - INFO - 2021-12-01 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,516 - [fut_strategy.py:373] - ultron - INFO - 2021-12-02 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,563 - [fut_strategy.py:373] - ultron - INFO - 2021-12-03 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,612 - [fut_strategy.py:373] - ultron - INFO - 2021-12-06 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,658 - [fut_strategy.py:373] - ultron - INFO - 2021-12-07 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,703 - [fut_strategy.py:373] - ultron - INFO - 2021-12-08 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,750 - [fut_strategy.py:373] - ultron - INFO - 2021-12-09 00:00:00 re-balance: 35 codes
2022-09-27 19:07:09,797 - [fut_strate

2022-09-27 19:07:12,495 - [fut_strategy.py:373] - ultron - INFO - 2022-03-09 00:00:00 re-balance: 35 codes
2022-09-27 19:07:12,532 - [fut_strategy.py:303] - ultron - ERROR - 2022-03-09T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:12,544 - [fut_strategy.py:373] - ultron - INFO - 2022-03-10 00:00:00 re-balance: 35 codes
2022-09-27 19:07:12,582 - [fut_strategy.py:303] - ultron - ERROR - 2022-03-10T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:12,594 - [fut_strategy.py:373] - ultron - INFO - 2022-03-11 00:00:00 re-balance: 35 codes
2022-09-27 19:07:12,629 - [fut_strategy.py:303] - ultron - ERROR - 2022-03-11T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:12,640 - [fut_strategy.py:373] - ultron - INFO - 2022-03-14 00:00:00 re-balance: 35 codes
2022-09-27 19:07:12,677 - [fut_strategy.py:303] - ultron - ERROR - 2022-03-14T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:12,689 - [fut_strategy.py:373] - ultron - INFO - 2022-0

2022-09-27 19:07:14,827 - [fut_strategy.py:373] - ultron - INFO - 2022-05-24 00:00:00 re-balance: 35 codes
2022-09-27 19:07:14,875 - [fut_strategy.py:373] - ultron - INFO - 2022-05-25 00:00:00 re-balance: 35 codes
2022-09-27 19:07:14,923 - [fut_strategy.py:373] - ultron - INFO - 2022-05-26 00:00:00 re-balance: 35 codes
2022-09-27 19:07:14,969 - [fut_strategy.py:373] - ultron - INFO - 2022-05-27 00:00:00 re-balance: 35 codes
2022-09-27 19:07:15,015 - [fut_strategy.py:373] - ultron - INFO - 2022-05-30 00:00:00 re-balance: 35 codes
2022-09-27 19:07:15,061 - [fut_strategy.py:373] - ultron - INFO - 2022-05-31 00:00:00 re-balance: 35 codes
2022-09-27 19:07:15,107 - [fut_strategy.py:373] - ultron - INFO - 2022-06-01 00:00:00 re-balance: 35 codes
2022-09-27 19:07:15,154 - [fut_strategy.py:373] - ultron - INFO - 2022-06-02 00:00:00 re-balance: 35 codes
2022-09-27 19:07:15,201 - [fut_strategy.py:373] - ultron - INFO - 2022-06-06 00:00:00 re-balance: 35 codes
2022-09-27 19:07:15,248 - [fut_strate

#### 回测

In [25]:
match_df, ret_df, positions = strategy.run(params=params, rate=0.0)

2022-09-27 19:07:15,871 - [fut_strategy.py:312] - ultron - INFO - starting re-balance ...
2022-09-27 19:07:15,882 - [fut_strategy.py:330] - ultron - INFO - running setting finished ...
2022-09-27 19:07:15,914 - [fut_strategy.py:373] - ultron - INFO - 2021-01-04 00:00:00 re-balance: 34 codes
2022-09-27 19:07:15,950 - [fut_strategy.py:303] - ultron - ERROR - 2021-01-04T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:15,960 - [fut_strategy.py:373] - ultron - INFO - 2021-01-05 00:00:00 re-balance: 34 codes
2022-09-27 19:07:15,996 - [fut_strategy.py:303] - ultron - ERROR - 2021-01-05T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:16,007 - [fut_strategy.py:373] - ultron - INFO - 2021-01-06 00:00:00 re-balance: 34 codes
2022-09-27 19:07:16,042 - [fut_strategy.py:303] - ultron - ERROR - 2021-01-06T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:16,053 - [fut_strategy.py:373] - ultron - INFO - 2021-01-07 00:00:00 re-balance: 34 codes
2022-09-27 1

2022-09-27 19:07:18,899 - [fut_strategy.py:373] - ultron - INFO - 2021-04-13 00:00:00 re-balance: 35 codes
2022-09-27 19:07:18,943 - [fut_strategy.py:373] - ultron - INFO - 2021-04-14 00:00:00 re-balance: 35 codes
2022-09-27 19:07:18,988 - [fut_strategy.py:373] - ultron - INFO - 2021-04-15 00:00:00 re-balance: 35 codes
2022-09-27 19:07:19,032 - [fut_strategy.py:373] - ultron - INFO - 2021-04-16 00:00:00 re-balance: 35 codes
2022-09-27 19:07:19,078 - [fut_strategy.py:373] - ultron - INFO - 2021-04-19 00:00:00 re-balance: 35 codes
2022-09-27 19:07:19,123 - [fut_strategy.py:373] - ultron - INFO - 2021-04-20 00:00:00 re-balance: 35 codes
2022-09-27 19:07:19,169 - [fut_strategy.py:373] - ultron - INFO - 2021-04-21 00:00:00 re-balance: 35 codes
2022-09-27 19:07:19,217 - [fut_strategy.py:373] - ultron - INFO - 2021-04-22 00:00:00 re-balance: 35 codes
2022-09-27 19:07:19,263 - [fut_strategy.py:373] - ultron - INFO - 2021-04-23 00:00:00 re-balance: 35 codes
2022-09-27 19:07:19,308 - [fut_strate

2022-09-27 19:07:22,446 - [fut_strategy.py:373] - ultron - INFO - 2021-08-04 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,490 - [fut_strategy.py:373] - ultron - INFO - 2021-08-05 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,538 - [fut_strategy.py:373] - ultron - INFO - 2021-08-06 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,583 - [fut_strategy.py:373] - ultron - INFO - 2021-08-09 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,628 - [fut_strategy.py:373] - ultron - INFO - 2021-08-10 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,673 - [fut_strategy.py:373] - ultron - INFO - 2021-08-11 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,720 - [fut_strategy.py:373] - ultron - INFO - 2021-08-12 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,765 - [fut_strategy.py:373] - ultron - INFO - 2021-08-13 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,810 - [fut_strategy.py:373] - ultron - INFO - 2021-08-16 00:00:00 re-balance: 35 codes
2022-09-27 19:07:22,854 - [fut_strate

2022-09-27 19:07:26,024 - [fut_strategy.py:373] - ultron - INFO - 2021-11-30 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,072 - [fut_strategy.py:373] - ultron - INFO - 2021-12-01 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,121 - [fut_strategy.py:373] - ultron - INFO - 2021-12-02 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,168 - [fut_strategy.py:373] - ultron - INFO - 2021-12-03 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,215 - [fut_strategy.py:373] - ultron - INFO - 2021-12-06 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,263 - [fut_strategy.py:373] - ultron - INFO - 2021-12-07 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,311 - [fut_strategy.py:373] - ultron - INFO - 2021-12-08 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,358 - [fut_strategy.py:373] - ultron - INFO - 2021-12-09 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,404 - [fut_strategy.py:373] - ultron - INFO - 2021-12-10 00:00:00 re-balance: 35 codes
2022-09-27 19:07:26,448 - [fut_strate

2022-09-27 19:07:29,065 - [fut_strategy.py:303] - ultron - ERROR - 2022-03-09T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:29,077 - [fut_strategy.py:373] - ultron - INFO - 2022-03-10 00:00:00 re-balance: 35 codes
2022-09-27 19:07:29,113 - [fut_strategy.py:303] - ultron - ERROR - 2022-03-10T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:29,125 - [fut_strategy.py:373] - ultron - INFO - 2022-03-11 00:00:00 re-balance: 35 codes
2022-09-27 19:07:29,162 - [fut_strategy.py:303] - ultron - ERROR - 2022-03-11T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:29,174 - [fut_strategy.py:373] - ultron - INFO - 2022-03-14 00:00:00 re-balance: 35 codes
2022-09-27 19:07:29,211 - [fut_strategy.py:303] - ultron - ERROR - 2022-03-14T00:00:00.000000000 rebalance error: infeasible
2022-09-27 19:07:29,223 - [fut_strategy.py:373] - ultron - INFO - 2022-03-15 00:00:00 re-balance: 35 codes
2022-09-27 19:07:29,260 - [fut_strategy.py:303] - ultron - ERROR - 2022-

...

2022-09-27 19:07:33,103 - [fut_strategy.py:454] - ultron - INFO - 2021-06-03 00:00:00: turn over 0.4, returns 0.0049, benchmark returns 0.0139
2022-09-27 19:07:33,110 - [fut_strategy.py:454] - ultron - INFO - 2021-06-04 00:00:00: turn over 0.4, returns -0.0025, benchmark returns -0.0104
2022-09-27 19:07:33,117 - [fut_strategy.py:454] - ultron - INFO - 2021-06-07 00:00:00: turn over 0.4, returns -0.004, benchmark returns -0.0018
2022-09-27 19:07:33,124 - [fut_strategy.py:454] - ultron - INFO - 2021-06-08 00:00:00: turn over 0.4, returns -0.0035, benchmark returns -0.0036
2022-09-27 19:07:33,131 - [fut_strategy.py:454] - ultron - INFO - 2021-06-09 00:00:00: turn over 0.4, returns -0.0052, benchmark returns -0.0049
2022-09-27 19:07:33,138 - [fut_strategy.py:454] - ultron - INFO - 2021-06-10 00:00:00: turn over 0.4, returns 0.0059, benchmark returns 0.0146
2022-09-27 19:07:33,145 - [fut_strategy.py:454] - ultron - INFO - 2021-06-11 00:00:00: turn over 0.4, returns 0.0065, benchmark returns

2022-09-27 19:07:33,510 - [fut_strategy.py:454] - ultron - INFO - 2021-08-25 00:00:00: turn over 0.4, returns -0.0058, benchmark returns -0.0054
2022-09-27 19:07:33,517 - [fut_strategy.py:454] - ultron - INFO - 2021-08-26 00:00:00: turn over 0.4, returns -0.0033, benchmark returns -0.0077
2022-09-27 19:07:33,524 - [fut_strategy.py:454] - ultron - INFO - 2021-08-27 00:00:00: turn over 0.4, returns 0.006, benchmark returns 0.0065
2022-09-27 19:07:33,531 - [fut_strategy.py:454] - ultron - INFO - 2021-08-30 00:00:00: turn over 0.4, returns -0.0026, benchmark returns -0.0041
2022-09-27 19:07:33,538 - [fut_strategy.py:454] - ultron - INFO - 2021-08-31 00:00:00: turn over 0.4, returns 0.0005, benchmark returns -0.0014
2022-09-27 19:07:33,545 - [fut_strategy.py:454] - ultron - INFO - 2021-09-01 00:00:00: turn over 0.4, returns -0.0005, benchmark returns 0.0083
2022-09-27 19:07:33,552 - [fut_strategy.py:454] - ultron - INFO - 2021-09-02 00:00:00: turn over 0.4, returns 0.0056, benchmark returns

2022-09-27 19:07:33,917 - [fut_strategy.py:454] - ultron - INFO - 2021-11-24 00:00:00: turn over 0.4, returns 0.0003, benchmark returns 0.002
2022-09-27 19:07:33,924 - [fut_strategy.py:454] - ultron - INFO - 2021-11-25 00:00:00: turn over 0.4, returns -0.0017, benchmark returns 0.02
2022-09-27 19:07:33,931 - [fut_strategy.py:454] - ultron - INFO - 2021-11-26 00:00:00: turn over 0.4, returns 0.0045, benchmark returns 0.0118
2022-09-27 19:07:33,938 - [fut_strategy.py:454] - ultron - INFO - 2021-11-29 00:00:00: turn over 0.4, returns 0.0008, benchmark returns 0.0069
2022-09-27 19:07:33,945 - [fut_strategy.py:454] - ultron - INFO - 2021-11-30 00:00:00: turn over 0.4, returns -0.0009, benchmark returns -0.0056
2022-09-27 19:07:33,952 - [fut_strategy.py:454] - ultron - INFO - 2021-12-01 00:00:00: turn over 0.4, returns 0.0024, benchmark returns 0.0077
2022-09-27 19:07:33,959 - [fut_strategy.py:454] - ultron - INFO - 2021-12-02 00:00:00: turn over 0.4, returns -0.0017, benchmark returns 0.008

2022-09-27 19:07:34,326 - [fut_strategy.py:454] - ultron - INFO - 2022-02-22 00:00:00: turn over 0.4, returns 0.0009, benchmark returns 0.0004
2022-09-27 19:07:34,333 - [fut_strategy.py:454] - ultron - INFO - 2022-02-23 00:00:00: turn over 0.4, returns 0.0071, benchmark returns 0.0229
2022-09-27 19:07:34,340 - [fut_strategy.py:454] - ultron - INFO - 2022-02-24 00:00:00: turn over 1.6017, returns -0.0054, benchmark returns -0.0054
2022-09-27 19:07:34,347 - [fut_strategy.py:454] - ultron - INFO - 2022-02-25 00:00:00: turn over 1.0751, returns 0.0059, benchmark returns 0.0059
2022-09-27 19:07:34,354 - [fut_strategy.py:454] - ultron - INFO - 2022-02-28 00:00:00: turn over 0.8148, returns 0.0165, benchmark returns 0.0165
2022-09-27 19:07:34,361 - [fut_strategy.py:454] - ultron - INFO - 2022-03-01 00:00:00: turn over 0.5426, returns 0.0035, benchmark returns 0.0035
2022-09-27 19:07:34,368 - [fut_strategy.py:454] - ultron - INFO - 2022-03-02 00:00:00: turn over 0.6616, returns 0.011, benchmar

2022-09-27 19:07:34,726 - [fut_strategy.py:454] - ultron - INFO - 2022-05-19 00:00:00: turn over 0.4, returns 0.0031, benchmark returns -0.005
2022-09-27 19:07:34,733 - [fut_strategy.py:454] - ultron - INFO - 2022-05-20 00:00:00: turn over 0.4, returns -0.0009, benchmark returns 0.0043
2022-09-27 19:07:34,740 - [fut_strategy.py:454] - ultron - INFO - 2022-05-23 00:00:00: turn over 0.4, returns 0.0008, benchmark returns 0.0046
2022-09-27 19:07:34,747 - [fut_strategy.py:454] - ultron - INFO - 2022-05-24 00:00:00: turn over 0.4, returns -0.0002, benchmark returns 0.0055
2022-09-27 19:07:34,754 - [fut_strategy.py:454] - ultron - INFO - 2022-05-25 00:00:00: turn over 0.4, returns 0.0091, benchmark returns 0.0038
2022-09-27 19:07:34,761 - [fut_strategy.py:454] - ultron - INFO - 2022-05-26 00:00:00: turn over 0.4, returns -0.0006, benchmark returns 0.0112
2022-09-27 19:07:34,768 - [fut_strategy.py:454] - ultron - INFO - 2022-05-27 00:00:00: turn over 0.4, returns -0.0044, benchmark returns -0